# MATRIZ DE INCIDENCIA

In [1]:
#Modulos
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Godel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1- Definir stoplist

In [13]:
with open("stoplist.txt", encoding="utf-8") as file:
    stoplist = [line.strip() for line in file]
stoplist += ['?', '-', '.', ':', ',', '!', ';']  

2-  Tokenizar y eliminar stopwords

In [14]:
def preprocesamiento(doc):
    file = open(doc, "r")
    # 1 - tokenizar
    words = nltk.word_tokenize(file.read())
    # 2 - eliminar stopwords
    words = [word for word in words if word not in stoplist]
    return words

3- Construir la matriz de incidencia

In [25]:
#use pandas for better visualization
import pandas as pd

def construir_matriz(list_docs):
    matriz = [[]]
    for libro in list_docs:
        words = preprocesamiento(libro)
        for word in words:
            if word.lower() not in matriz[0]:
                matriz[0].append(word.lower())
                for i in range(1, len(matriz)):
                    matriz[i].append(0)
        matriz.append([0]*len(matriz[0]))
        for word in words:
            matriz[-1][matriz[0].index(word.lower())] = 1
    return matriz

def construir_matriz_DF(list_docs):
    matriz = [[]]
    for libro in list_docs:
        words = preprocesamiento(libro)
        for word in words:
            if word.lower() not in matriz[0]:
                matriz[0].append(word.lower())
                for i in range(1, len(matriz)):
                    matriz[i].append(0)
        matriz.append([0]*len(matriz[0]))
        for word in words:
            matriz[-1][matriz[0].index(word.lower())] = 1
    return pd.DataFrame(matriz[1:], columns=matriz[0], index=[f"libro{i}" for i in range(1, len(matriz))])


construir_matriz_DF(["libro1.txt", "libro2.txt", "libro3.txt", "libro4.txt", "libro5.txt", "libro6.txt"])

,la,obra,comienza,noticia,celebraciã³n,111âº,cumpleaã±os,bilbo,bolsã³n,comarca,...,juntos,viajan,puertos,grises,esperaban,cã­rdan,dolorosa,despedida,barco,imperecederas
libro1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
libro2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
libro3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
libro4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
libro5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
libro6,0,0,1,0,0,0,0,1,0,1,...,1,1,1,1,1,1,1,1,1,1


4- Aplicar consultas booleanas

In [44]:
list_docs = ["libro1.txt", "libro2.txt", "libro3.txt", "libro4.txt", "libro5.txt", "libro6.txt"]
matriz = construir_matriz(list_docs)
matriz_DF = construir_matriz_DF(list_docs)

def M(word):
    lista = []
    if word.lower() not in matriz[0]:
        return []
    for i in range(1, len(matriz)):
        lista.append(matriz[i][matriz[0].index(word)])
        
    l = []
    
    for i in range(len(lista)):
        if lista[i] > 0:
            l.append(list_docs[i])
    return l

def M_DF(word):
    if word.lower() not in matriz_DF.columns:
        return []
    return matriz_DF[matriz_DF[word.lower()] == 1].index
    
def AND(A, B):
    return [i for i in A if i in B]

def OR(A, B):
    return A + list(set(B) - set(A)) # por teoria de conjuntos A U B = A + (B - A)

def NOT(A):
    list =[i for i in list_docs if i not in A]
    return list

Query1 = AND(M("anillo"), M("frodo"))
print("anillo AND frodo: ", Query1)

Query2 = OR(M("anillo"), M("frodo"))
print("anillo OR frodo: ", Query2)

# definir mas queries
Query3 = NOT(M("anillo"))
print("NOT anillo: ", Query3)

print("=="*50)

anillo AND frodo:  ['libro1.txt', 'libro2.txt', 'libro4.txt', 'libro6.txt']
anillo OR frodo:  ['libro1.txt', 'libro2.txt', 'libro4.txt', 'libro6.txt', 'libro3.txt', 'libro5.txt']
NOT anillo:  ['libro3.txt', 'libro5.txt']
